In [3]:
import pandas as pd
import numpy as np
import math
import operator

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import confusion_matrix
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV

import seaborn as sns
import statsmodels.api as sm
import matplotlib.pyplot as plt

# Exploratory Data Analysis

# Training Set

In [4]:
# Load Aggregated Signature from 12 Loc as Training Set
trainset = pd.read_csv('./Datasets/SensorLocDS/12LocAggregated.csv')
print(trainset.head())
lastIndex = len(trainset)
#dataset.iloc[rowrange, columnrange]
ips_data = trainset.iloc[:lastIndex, 1:12]
ips_labels = trainset.iloc[:lastIndex, :1]
k_value = math.ceil(math.sqrt(len(ips_data)))
X = ips_data.to_numpy()
y = ips_labels.to_numpy()
X_train = X
y_train = y
y_train = y_train.reshape(len(y_train),)
print("y_train: ", y_train)
# X_train, X_test, y_train, y_test = train_test_split(X.to_numpy(), y.to_numpy(), shuffle = True,test_size=0.3) 

  Row Labels      0x0001      0x0002      0x0003      0x0004      0x0005  \
0        1,1  193.201814  189.459016  183.521739  182.930175  177.560554   
1       1,13  179.375297  184.668258  190.644342  177.951872  178.179916   
2        1,7  180.799097  193.402954  191.345992  177.638821  183.760234   
3       13,1  177.000000  177.031056  177.912913  189.466667  178.398625   
4      13,13  177.000000  177.997436  178.856643  178.914110  180.496914   

       0x0006      0x0007      0x0008      0x0009      0x000A      0x000B  \
0  177.000000  177.369458  177.735484  177.000000  177.000000  177.000000   
1  180.367491  178.086207  177.000000  177.003846  177.000000  177.885650   
2  184.237251  177.333333  180.013044  178.722741  177.018182  179.716904   
3  178.428969  192.546667  183.521964  180.256471  180.503185  187.321348   
4  188.444444  177.009174  179.334963  195.192825  177.465201  183.170306   

       0x000C  
0  177.000000  
1  180.515152  
2  184.456897  
3  179.300836  


# Resample to Create Test Set

In [8]:
# FIX ISSUE - QUERIED ON DIFF DATES -> MORE THAN 24 HOUR DIFFERENCE!!!!
        
csvfile = pd.read_csv('./Datasets/SensorLocDS/Query12LocALL.csv')
df = pd.DataFrame(csvfile)
# print(df.head())
df.columns = ['Time','ID2','RSSI','Location']
# print(df.info())
df['Time'] = df['Time'].str.strip()
df['Time'] = pd.to_datetime(df['Time'])
df['Time'].date='20-03-15'
# print("CODE: 1A2B\n", df)

from datetime import timedelta
print("start time: ", df['Time'].iloc[0], "\t", "end time: ", df['Time'].iloc[-1])
timeDur = ( (df['Time'].iloc[-1] - df['Time'].iloc[0]).seconds / 2 )
df['ID2'] = df['ID2'].astype(str)
df['ID2'] = df['ID2'].apply(int, base=16)
sampleStartTime = df['Time'][0] + timedelta(seconds=2)


sortdf = df.copy()
sortdf.sort_values(by=['ID2'], inplace=True)

print("CODE: 1T4Z\n", df.head(15))
height =int(timeDur)
print("height: ", height)
resampled = pd.DataFrame(0, index=range(2*height), columns=sortdf['ID2'].unique())
resampled['Location'] = -1
for columns in resampled:
    resampled[columns] = 0
    

print(resampled.head())
#resampled.iloc[index, column] = value;

from collections import Counter
sensorSamplesCount = Counter()
ctr=0
for record in df.values:
# record[0] - timestamp, record[1] - ID2, record[2] - RSSI, record[3] - Location
    dfTimestamp = record[0]
    dfDeviceID = record[1]
    dfRSSI = record[2]
    dfLocation = record[3]
    if (record[0] <= sampleStartTime):
        sensorSamplesCount[dfDeviceID] += 1
        #dataset.iloc[rowrange, columnrange]
        resampled.iloc[ctr, dfDeviceID-1] = (resampled.iloc[ctr, dfDeviceID-1] + dfRSSI) / sensorSamplesCount[dfDeviceID] 
        resampled.iloc[ctr, -1] = dfLocation #fill location
    else:
        ctr=ctr+1
        sensorSamplesCount = Counter() #reset sensor sample counts
        sampleStartTime = dfTimestamp + timedelta(seconds=2)
        sensorSamplesCount[dfDeviceID] += 1
        resampled.iloc[ctr, dfDeviceID-1] = (resampled.iloc[ctr, dfDeviceID-1] + dfRSSI) / sensorSamplesCount[dfDeviceID] 
        resampled.iloc[ctr, -1] = dfLocation #fill label

# lastIndex = (resampled['Location'] == 0).argmax() #find first non-valid record and record index


writeResample = False
if (writeResample == True):
    resampled.to_csv('./Datasets/SensorLocDS/resampled12Loc.csv', index=False) 
    print("FILE WRITTEN!")
    
resampled.head(100)

start time:  2020-03-23 15:14:49 	 end time:  2020-03-23 15:48:51
CODE: 1T4Z
                   Time  ID2  RSSI Location
0  2020-03-23 15:14:49    1   192      1,1
1  2020-03-23 15:14:49    3   177      1,1
2  2020-03-23 15:14:49    4   179      1,1
3  2020-03-23 15:14:50    2   189      1,1
4  2020-03-23 15:14:51    8   177      1,1
5  2020-03-23 15:14:51    5   177      1,1
6  2020-03-23 15:14:51    1   193      1,1
7  2020-03-23 15:14:51   11   177      1,1
8  2020-03-23 15:14:51    9   177      1,1
9  2020-03-23 15:14:51    3   180      1,1
10 2020-03-23 15:14:52    2   182      1,1
11 2020-03-23 15:14:53    8   177      1,1
12 2020-03-23 15:14:53    1   191      1,1
13 2020-03-23 15:14:53    3   185      1,1
14 2020-03-23 15:14:53   12   177      1,1
height:  1021
   1  2  3  4  5  6  7  8  9  10  11  12  Location
0  0  0  0  0  0  0  0  0  0   0   0   0         0
1  0  0  0  0  0  0  0  0  0   0   0   0         0
2  0  0  0  0  0  0  0  0  0   0   0   0         0
3  0  0  0  0  0

TypeError: Cannot compare type Timedelta with type int

# Test Set

In [9]:
testset = pd.read_csv('./Datasets/SensorLocDS/Query12LocALL.csv')
print(testset.head())
lastIndex = len(testset)
#dataset.iloc[rowrange, columnrange]
ips_data = testset.iloc[0:lastIndex, :11]
ips_labels = testset.iloc[:lastIndex, 11:]
X = ips_data.to_numpy()
y = ips_labels.to_numpy()
X_test = X
y_test = y
y_train = y_train.reshape(len(y_train),)
print("y_test: ", y_test)

# X_train, X_test, y_train, y_test = train_test_split(X.to_numpy(), y.to_numpy(), shuffle = True,test_size=0.3) 

      Date       Time      ID2  \
0  16/3/20   15:14:49   0x0001   
1  16/3/20   15:14:49   0x0003   
2  16/3/20   15:14:49   0x0004   
3  16/3/20   15:14:50   0x0002   
4  16/3/20   15:14:51   0x0008   

                                            Raw Data  RSSI Location  
0   0201041AFF4C0002150005000100001000800000805F9...   192      1,1  
1   0201041AFF4C0002150005000100001000800000805F9...   177      1,1  
2   0201041AFF4C0002150005000100001000800000805F9...   179      1,1  
3   0201041AFF4C0002150005000100001000800000805F9...   189      1,1  
4   0201041AFF4C0002150005000100001000800000805F9...   177      1,1  
y_test:  []


# Parameter Optimisation

# KNN Algorithm - Distance Calculation

In [ ]:
n_samples = 2

#Function calculates euclidean distance between two n-dimensional data instances 
def euclideanDistance(instance1, instance2):
    #handles if instances are lists or tuples:
    instance1 = np.array(instance1) 
    instance2 = np.array(instance2)
    
    '''
    https://docs.scipy.org/doc/numpy/reference/generated/numpy.linalg.norm.html
    uses 2-norm frobenius norm and returns euclidean distance
    '''
    return np.linalg.norm(instance1 - instance2) #euclidean distance

# KNN Algorithm - Determining Neighbours

In [ ]:
#Function finds nearest neighbours; nearest -> smallest euclidean distance
def get_neighbors(training_set, 
                  labels, 
                  test_instance, 
                  k, 
                  distance=euclideanDistance):
    """
    get_neighbors calculates a list of the k nearest neighbors
    of an instance 'test_instance'.
    The list neighbors contains 3-tuples with  
    (index, dist, label)
    where
    index    is the index from the training_set, 
    dist     is the distance between the test_instance and the 
             instance training_set[index]
    distance is a reference to a function used to calculate the 
             distances
    """
    distances = [] #empty distance array
    
    #calculates euclidean distance between test_instance and ALL other instances in training_set
    for index in range(len(training_set)):
        dist = euclideanDistance(test_instance, training_set[index])
        distances.append((training_set[index], dist, labels[index]))
    distances.sort(key=lambda x: x[1])
    neighbors = distances[:k]
    return neighbors # The list neighbors contains 3-tuples with (index, dist, label)

In [ ]:
# Get Neighbours
outArray = []
for i in range(n_samples):
    neighbors = get_neighbors(X_train, 
                              y_train, 
                              X_test[i], 
                              k_value, 
                              distance=euclideanDistance)
#     print(i,
#           X_test[i],
#           y_test[i],
#           neighbors)
    
    outArray.append([i,
          X_test[i],
          y_test[i],
          neighbors])

out_df = pd.DataFrame(outArray, columns=['i', 'X_test', 'y_test', 'neighbours'])
out_df.head()
# out_df.tail()

In [ ]:
from collections import Counter
#LOC: outputs for each [x,y] in label, 
#     where (x_n1, y_n1) rep x and y coord of 1st neighbour resp
#     x_out = sum(x_n1, ... x_N) / N, where total neighbours = N
def calcLocation(neighbours): 
    x_out = 0
    y_out = 0
    totalNeighbours = len(neighbours)
    for neighbour in neighbours:
        # handle label (str) to tuple(int, int)
        labeltuple = neighbour[2]
        labeltuple = tuple(map(int, labeltuple.split(',')))
#         print("neighbour label: ", labeltuple)
        x_nb = labeltuple[0]
        y_nb = labeltuple[1]
#         print("x_nb: ", x_nb)
#         print("y_nb: ", y_nb)
        x_out += x_nb
        y_out += y_nb
#         print("x_out: ", x_out)
#         print("y_out: ", y_out)
    return ((x_out/totalNeighbours) , (y_out/totalNeighbours))
    


In [ ]:
print("NON-WEIGHTED POSITION KNN\n")
for i in range(n_samples):
    neighbors = get_neighbors(X_train, 
                              y_train, 
                              X_test[i], 
                              k_value, 
                              distance=euclideanDistance)
    testPosTuple = tuple(map(int, y_test[i][0].split(',')))
    print("test location: ", testPosTuple)
    print("calculalated location: ", calcLocation(neighbors))
    print("distance error: ", euclideanDistance(testPosTuple, calcLocation(neighbors)))
    print("----------------------------")
    


In [ ]:
#Function returns vote 'probability' - i.e. distribution/percentage majority vote
def vote_prob(neighbors):
    class_counter = Counter() # Counter object - https://docs.python.org/2/library/collections.html
    for neighbor in neighbors:
        class_counter[neighbor[2]] += 1 #add to count of target (class)
        
    # aggregates into tuples ~ zip(*iterables), 
    # Return a list of the n most common elements and their counts from the most common to the least.    
    labels, votes = zip(*class_counter.most_common()) #returns list of sorted most common [labels], [votes]
    #print("L|V: ", labels, votes)
    #print("Class Counter: ", class_counter.most_common)
    winner = class_counter.most_common(1)[0][0]       #majority label
    votes4winner = class_counter.most_common(1)[0][1] #majority vote count
    return winner, votes4winner/sum(votes)            #returns majority label, majority proportion

In [ ]:
resultsKNN = pd.DataFrame(0, index=range(len(X_test)), columns=['index', 'data', 'pred_location', 'label', 'error'])

indexArray = []
dataArray = []
locArray = []
labelArray = []
errorArray = []
for i in range(len(X_test)):
    neighbors = get_neighbors(X_train, 
                              y_train, 
                              X_test[i], 
                              k_value, 
                              distance=euclideanDistance)
    
    testPosTuple = tuple(map(int, y_test[i][0].split(',')))
#     print("test location: ", testPosTuple)
#     print("calculalated location: ", calcLocation(neighbors))
#     print("distance error: ", euclideanDistance(testPosTuple, calcLocation(neighbors)))
    
    indexArray.append(i)
    dataArray.append(X_test[i])
    locArray.append(calcLocation(neighbors))
    labelArray.append(y_test[i][0])
    errorArray.append(euclideanDistance(testPosTuple, calcLocation(neighbors)))


resultsKNN['index'] = indexArray
resultsKNN['data'] = dataArray
resultsKNN['pred_location'] = locArray
resultsKNN['label'] = labelArray 
resultsKNN['error'] = errorArray


resultsKNN.to_csv('./Datasets/SensorLocDS/Aggregated/KNN_LOC_AGG.csv', index=False)
print("FILE WRITTEN")
resultsKNN.head(20)

# Weighted KNN

In [ ]:
'''
def calcLocationRankW(neighbours): 
    x_out = 0
    y_out = 0
    rank = 0
    for neighbour in neighbours:
        rank += 1
        # handle label (str) to tuple(int, int)
        labeltuple = neighbour[2]
        labeltuple = tuple(map(int, labeltuple.split(',')))
#         print("neighbour label: ", labeltuple)
        x_nb = labeltuple[0]
        y_nb = labeltuple[1]
#         print("x_nb: ", x_nb)
#         print("y_nb: ", y_nb)
        x_out += x_nb*(1/rank)
        y_out += y_nb*(1/rank)
#         print("x_out: ", x_out)
#         print("y_out: ", y_out)
    return ((x_out) , (y_out))
'''

In [ ]:
"""
resultsRWKNN = pd.DataFrame(0, index=range(len(X_test)), columns=['index', 'data', 'label', 'vote_result', 'prediction'])

indexArray = []
dataArray = []
labelArray = []
voteArray = []
predArray = []

for i in range(len(X_test)):
    neighbors = get_neighbors(X_train, 
                              y_train, 
                              X_test[i], 
                              k_value, 
                              distance=euclideanDistance)
    
    testPosTuple = tuple(map(int, y_test[i][0].split(',')))
    print("test location: ", testPosTuple)
    print("calculalated location: ", calcLocationRankW(neighbors))
    print("distance error: ", euclideanDistance(testPosTuple, calcLocation(neighbors)))
    
# resultsRWKNN['index'] = indexArray
# resultsRWKNN['data'] = dataArray
# resultsRWKNN['label'] = labelArray
# resultsRWKNN['vote_result'] = voteArray
# resultsRWKNN['prediction'] = predArray


# resultsRWKNN.to_csv('./Datasets/Testing/RANKKNN_LOC.csv', index=False) 
# print("FILE WRITTEN")
# resultsRWKNN
"""


# Distance Weighted KNN

In [ ]:
def calcLocationDistW(neighbours): 
    x_out = 0
    y_out = 0
    total_dist = 0
    for neighbour in neighbours:
        # handle label (str) to tuple(int, int)
        labeltuple = neighbour[2]
        labeltuple = tuple(map(int, labeltuple.split(',')))
        # distance
        dist_nb = neighbour[1]
        total_dist += dist_nb
#         print("Neighbour distance: ", dist_nb)
#         print("Total distance: ", total_dist)
        x_nb = labeltuple[0]
        y_nb = labeltuple[1]
#         print("x_nb: ", x_nb)
#         print("y_nb: ", y_nb)
        x_out += x_nb*(dist_nb)
        y_out += y_nb*(dist_nb)
#         print("x_out: ", x_out)
#         print("y_out: ", y_out)
    return ((x_out/total_dist) , (y_out/total_dist))


In [ ]:
for i in range(len(X_test)):
    neighbors = get_neighbors(X_train, 
                              y_train, 
                              X_test[i], 
                              k_value, 
                              distance=euclideanDistance)
    
    testPosTuple = tuple(map(int, y_test[i][0].split(',')))
    print("test location: ", testPosTuple)
    print("calculalated location: ", calcLocationDistW(neighbors))
    print("distance error: ", euclideanDistance(testPosTuple, calcLocationDistW(neighbors)))
    print("---------------------------------------------------------")

In [ ]:
resultsDWKNN = pd.DataFrame(0, index=range(len(X_test)), columns=['index', 'data', 'pred_location', 'label', 'error'])

writeFile = True
indexArray = []
dataArray = []
locArray = []
labelArray = []
errorArray = []
for i in range(len(X_test)):
    neighbors = get_neighbors(X_train, 
                              y_train, 
                              X_test[i], 
                              k_value, 
                              distance=euclideanDistance)
    
    testPosTuple = tuple(map(int, y_test[i][0].split(',')))
#     print("test location: ", testPosTuple)
#     print("calculalated location: ", calcLocationDistW(neighbors))
#     print("distance error: ", euclideanDistance(testPosTuple, calcLocationDistW(neighbors)))
    
    indexArray.append(i)
    dataArray.append(X_test[i])
    locArray.append(calcLocationDistW(neighbors))
    labelArray.append(y_test[i][0])
    errorArray.append(euclideanDistance(testPosTuple, calcLocationDistW(neighbors)))


resultsDWKNN['index'] = indexArray
resultsDWKNN['data'] = dataArray
resultsDWKNN['pred_location'] = locArray
resultsDWKNN['label'] = labelArray 
resultsDWKNN['error'] = errorArray

if (writeFile == True):
    resultsDWKNN.to_csv('./Datasets/SensorLocDS/Aggregated/DWKNN_LOC_AGG.csv', index=False)
    print("FILE WRITTEN")
    
resultsDWKNN.head(20)

In [ ]:
DataDWKNN = pd.read_csv('./Datasets/SensorLocDS/Aggregated/DWKNN_LOC_AGG.csv')
# Find the unique values

# unique_values = np.sort(pd.Series)
# Make the rank array for these sorted and unique values in the dataset -
# ranks = np.arange(0,len(unique_values))/(len(unique_values)-1)
# Plot unique_values vs ranks

error = DataDWKNN['error']
## sort the unique values using pandas unique function
dwknn_error = np.sort(error.unique())
dwknn_cdf = np.arange(0,len(dwknn_error),step=1)/(len(dwknn_error)-1)

## plotting    
plt.plot(dwknn_error,dwknn_cdf)
plt.show()

In [ ]:
DataKNN = pd.read_csv('./Datasets/SensorLocDS/Aggregated/KNN_LOC_AGG.csv')
# Find the unique values

# unique_values = np.sort(pd.Series)
# Make the rank array for these sorted and unique values in the dataset -
# ranks = np.arange(0,len(unique_values))/(len(unique_values)-1)
# Plot unique_values vs ranks

error = DataKNN['error']
## sort the unique values using pandas unique function
knn_error = np.sort(error.unique())
knn_cdf = np.arange(0,len(knn_error),step=1)/(len(knn_error)-1)

## plotting    
plt.plot(knn_error,knn_cdf)
plt.show()

In [ ]:
DataDWKNN = pd.read_csv('./Datasets/SensorLocDS/Aggregated/DWKNN_LOC_AGG.csv')
# Find the unique values

# unique_values = np.sort(pd.Series)
# Make the rank array for these sorted and unique values in the dataset -
# ranks = np.arange(0,len(unique_values))/(len(unique_values)-1)
# Plot unique_values vs ranks

error = DataDWKNN['error']
## sort the unique values using pandas unique function
dwknn_error = np.sort(error.unique())
cdf = np.arange(0,len(dwknn_error),step=1)/(len(dwknn_error)-1)

## plotting    
plt.plot(dwknn_error,cdf, "blue")


DataKNN = pd.read_csv('./Datasets/SensorLocDS/Aggregated/KNN_LOC_AGG.csv')
# Find the unique values

# unique_values = np.sort(pd.Series)
# Make the rank array for these sorted and unique values in the dataset -
# ranks = np.arange(0,len(unique_values))/(len(unique_values)-1)
# Plot unique_values vs ranks

error = DataKNN['error']
## sort the unique values using pandas unique function
knn_error = np.sort(error.unique())
cdf = np.arange(0,len(knn_error),step=1)/(len(knn_error)-1)

## plotting    
plt.plot(knn_error,cdf, "red")
plt.show()

In [ ]:
t = np.linspace(0, 2*math.pi, 400)
a = np.sin(t)
b = np.cos(t)
c = a + b

plt.plot(t, a, 'r') # plotting t, a separately 
plt.plot(t, b, 'b') # plotting t, b separately 
plt.plot(t, c, 'g') # plotting t, c separately 
plt.show()